In [1]:
import pandas as pd
import re
from pymorphy3 import MorphAnalyzer

In [2]:
with open('data1.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

In [3]:
def open_json(file_name):
    with open(file_name, encoding='utf-8') as inputfile:
        df = pd.read_json(inputfile)
    return df
    

In [4]:
df1= open_json('data20.json')

In [9]:
data = data.drop_duplicates(ignore_index = True)
data = data.dropna()

In [11]:
data.to_csv('general_data.csv')

'Опыт работы\n11 месяцев\nАвгуст 2023 — Июнь 2024\nКосметологическая клиника LOYALTY, Косметолог\nПроведение процедур лазерной эпиляции (александритовый, диодный лазер), чистки, пилинги, биоревитализация, аппаратная косметология\n1 год и 6 месяцев\nЯнварь 2022 — Июнь 2023\nСамозанятость, Косметолог/мастер лазерной эпиляции\nЛазерная эпиляция, косметология\nУчебные заведения\nГод окончания: 2023\nМедицинский колледж 1\nСестринское дело\nМедкнижка\nМедкнижка — Есть\nГражданство\nГражданство — Россия'

In [10]:
data

,num,table,basic,description
0,№ 3317133582,Учебные заведения\nГод окончания: 2014\nММК им...,"Тип занятости: Основная работа, Подработка\nСф...",Опыт работы в косметологии менее года
1,№ 4585551392,Гражданство\nГражданство — Россия,Может выйти завтра: Нет\nТип занятости: Подраб...,
2,№ 3628798315,Учебные заведения\nГод окончания: 2004\nБрянск...,Тип занятости: Основная работа\nГотовность к к...,"Эстeтичеcкая, инъекциoннaя, аппаратнaя коcмето..."
3,№ 4525922612,Учебные заведения\nГод окончания: 2008\nЧитинс...,Тип занятости: Подработка\nСфера деятельности:...,—
4,№ 4600063172,Учебные заведения\nГод окончания: 2024\nUltra ...,"Тип занятости: Основная работа, Подработка\nСф...","Mеня зoвут Анна, я нa дaнный момент работaю в ..."
...,...,...,...,...
1194,№ 3241635338,Опыт работы\n8 лет и 6 месяцев\nИюль 2016 — на...,Тип занятости: Основная работа\nГотовность к к...,"Cpедние cпeциальнoе образовaниe, прохoдила пов..."
1195,№ 614211068,Опыт работы\n5 месяцев\nАвгуст 2024 — настояще...,Может выйти завтра: Нет\nТип занятости: Основн...,"Уверенный пользователь ПК, ответственная."
1196,№ 3057007196,Опыт работы\n10 месяцев\nСентябрь 2022 — Июнь ...,Тип занятости: Основная работа\nГотовность к к...,Мнe вceгдa нрaвилось заниматься кoжей и eе здo...
1197,№ 2568568827,Опыт работы\n3 года и 2 месяца\nОктябрь 2019 —...,"Сфера деятельности: Фитнес, салоны красоты\nГр...","Коммуникабельная,ответственная,аккуратная и оч..."


## Обработка опыта работы

In [14]:
def find_between_strings(lst, start_str, end_str):
    result = []
    found_start = False
    lst = lst.split('\n')
    for item in lst:
        if found_start:
            if item == end_str:
                break
            result.append(item)
        elif item == start_str:
            found_start = True
    if result == []:
        return 'Не указано'
    else:
        return result
        

In [15]:
start_word = "Опыт работы"
end_word = "Учебные заведения"
data["Опыт работы"] = data.table.apply(find_between_strings, args=(start_word, end_word))

In [ ]:
morpher = MorphAnalyzer()

In [17]:
def preprocessing(df):
    text = df.lower()
    text = (
        ' '.join([
            morpher.parse(word)[0].normal_form 
            for word in text.split()]))
    return text

In [18]:
cosmetologist_words = ['косметолог','косметология', 'эстетист', 'косметолог-эстетист', 'медсестра-косметолог', 'врач', 'врач-косметолог', 'дерматовенеролог', 'лаборатория', 'лаборант']

In [19]:
# Функция для преобразования строки с длительностью в месяцы
def convert_to_months(date):
    months = 0
    if 'год' in preprocessing(date):
        years = int(re.search(r'(\d+)\s*год', preprocessing(date)).group(1))
        months += years * 12
    # Если указано "месяц" или "месяцев", добавляем указанные месяцы
    if 'месяц' in preprocessing(date):
        months_part = re.search(r'(\d+)\s*месяц', preprocessing(date))
        if months_part:
            months += int(months_part.group(1))
    return months

In [21]:
#Функция преобразования опыта работы
def analyze_experience(df):
    total_years = 0
    cosmetologist_jobs = []
    if df == 'Не указано':
        return 'Не указано'
    else:
        i = 0
        while i < len(df):
            if i + 3 >= len(df):
                break
            months = convert_to_months(df[i])
            years = months / 12
            years = round(years,2)
            job_title = df[i + 2] # название компании
            job_describe = df[i+3] # описаие обязанностей
            job_title_lemma = preprocessing(job_title)
            for word in cosmetologist_words:
                if word in job_title_lemma:
                    cosmetologist_jobs.append(f'year*{years}; company*{job_title}; responsibilities*{job_describe}')
                    total_years += years
            i+=4
    cosmetologist_set = set(cosmetologist_jobs)
    return cosmetologist_set



In [23]:
data['Опыт работы'] = data['Опыт работы'].apply(analyze_experience)

In [24]:
data['Опыт работы'].iloc[14]

{'year*0.92; company*Косметологическая клиника LOYALTY, Косметолог; responsibilities*Проведение процедур лазерной эпиляции (александритовый, диодный лазер), чистки, пилинги, биоревитализация, аппаратная косметология',
 'year*1.5; company*Самозанятость, Косметолог/мастер лазерной эпиляции; responsibilities*Лазерная эпиляция, косметология'}

In [25]:
#Функция разделения словаря
def process_experience(data):
    total_year = 0  # сумма всех year
    companies = set()  # компании
    responsibilities = set()  #обязанностей
    if data ==  'Не указано' or data == {}:
        return None
    else:
        for experience_str in data:
            parts = experience_str.split(';')

            year = None
            company = None
            responsibility = None

            for part in parts:
                if 'year' in part:
                    year = float(part.split('*')[1].strip())  # Извлекаем и приводим year к числу
                    total_year += year  # Добавляем к общей сумме
                elif 'company' in part:
                    company = part.split('*')[1].strip()  # Извлекаем компанию
                    companies.add(company)  # Добавляем компанию в множество
                elif 'responsibilities' in part:
                    responsibility = part.split('*')[1].strip()  # Извлекаем обязанности
                    responsibilities.add(responsibility)  # Добавляем обязанности в множество

        company_list = list(companies)
        responsibility_list = list(responsibilities)
        d = {
        'Стаж работы': total_year,
        'Компания': company_list,
        'Обязанности': responsibility_list
         }
        return d


In [26]:
result = data['Опыт работы'].apply(process_experience)

In [27]:
experience = pd.json_normalize(result)

In [29]:
experience

,Стаж работы,Компания,Обязанности
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1099,11.25,"[Частная, Врач косметолог]",[Инъекционная косметология от а до я .]
1100,0.83,"[Салон красоты «ЭКЗОТИКА», Косметолог-эстетист]","[Эстетические процедуры ,такие как : чистки , ..."
1101,3.17,"[Студия красоты «МАсква», Визажист-бровист,кос...",[Оказание косметологических услуг клиентам: - ...
1102,NaN,NaN,NaN


In [30]:
data.reset_index(drop=True, inplace=True)

In [31]:
data = pd.concat([data,experience], axis=1) #соединяем с исходным датасетом

In [32]:
data

,num,table,basic,description,Опыт работы,Стаж работы,Компания,Обязанности
0,№ 3317133582,Учебные заведения\nГод окончания: 2014\nММК им...,"Тип занятости: Основная работа, Подработка\nСф...",Опыт работы в косметологии менее года,Не указано,NaN,NaN,NaN
1,№ 4585551392,Гражданство\nГражданство — Россия,Может выйти завтра: Нет\nТип занятости: Подраб...,,Не указано,NaN,NaN,NaN
2,№ 3628798315,Учебные заведения\nГод окончания: 2004\nБрянск...,Тип занятости: Основная работа\nГотовность к к...,"Эстeтичеcкая, инъекциoннaя, аппаратнaя коcмето...",Не указано,NaN,NaN,NaN
3,№ 4525922612,Учебные заведения\nГод окончания: 2008\nЧитинс...,Тип занятости: Подработка\nСфера деятельности:...,—,Не указано,NaN,NaN,NaN
4,№ 4600063172,Учебные заведения\nГод окончания: 2024\nUltra ...,"Тип занятости: Основная работа, Подработка\nСф...","Mеня зoвут Анна, я нa дaнный момент работaю в ...",Не указано,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1099,№ 4381095053,Опыт работы\n11 лет и 3 месяца\nОктябрь 2013 —...,Тип занятости: Подработка\nСфера деятельности:...,Рассматриваю дополнительный прием пациентов то...,"{year*11.25; company*Частная, Врач косметолог;...",11.25,"[Частная, Врач косметолог]",[Инъекционная косметология от а до я .]
1100,№ 3057007196,Опыт работы\n10 месяцев\nСентябрь 2022 — Июнь ...,Тип занятости: Основная работа\nГотовность к к...,Мнe вceгдa нрaвилось заниматься кoжей и eе здo...,"{year*0.83; company*Салон красоты «ЭКЗОТИКА», ...",0.83,"[Салон красоты «ЭКЗОТИКА», Косметолог-эстетист]","[Эстетические процедуры ,такие как : чистки , ..."
1101,№ 2568568827,Опыт работы\n3 года и 2 месяца\nОктябрь 2019 —...,"Сфера деятельности: Фитнес, салоны красоты\nГр...","Коммуникабельная,ответственная,аккуратная и оч...","{year*3.17; company*Студия красоты «МАсква», В...",3.17,"[Студия красоты «МАсква», Визажист-бровист,кос...",[Оказание косметологических услуг клиентам: - ...
1102,№ 4222300635,Учебные заведения\nГод окончания: 2023\nШкола ...,Может выйти завтра: Да\nТип занятости: Основна...,Меня зовут Светлана\nВ данный момент работаю в...,Не указано,NaN,NaN,NaN


In [33]:
def explode_list(data):
    if data == "Не указано":
        pass
    else:
        return ','.join(data)

In [39]:
data['Компания'] = data['Компания'].apply(explode_list)

In [40]:
data['Обязанности'] = data['Обязанности'].apply(explode_list)

In [34]:
values = {'Стаж работы': 0,'Компания': "Не указано", 'Обязанности': "Не указано"}

In [42]:
data=data.fillna(value = values)

In [43]:
data['Обязанности'].iloc[14]

'Лазерная эпиляция, косметология,Проведение процедур лазерной эпиляции (александритовый, диодный лазер), чистки, пилинги, биоревитализация, аппаратная косметология'

## Обработка столбца table

In [44]:
start_word = "Учебные заведения"
end_word = "Медкнижка"
data["Учебные заведения"] = data.table.apply(find_between_strings, args=(start_word, end_word))

In [45]:
start_word = "Гражданство"
data['Гражданство'] = data.table.apply(find_between_strings, args=(start_word, None))

In [46]:
for elem in data['Гражданство']:
    if elem == ['Гражданство — Россия']:
        data['Гражданство'] = 'Россия'
    else: 
        data['Гражданство'] = 'другое'
  

In [47]:
def get_strings_between_start(lst):
    in_between = False  # Флаг, который указывает, находимся ли мы между строками с нужными префиксами
    result = []

    for line in lst:
        # Если строка начинается с start_prefix, начинаем собирать строки
        if line.startswith('Год окончания'):
            in_between = True
            continue  # Пропускаем строку с началом "start_prefix"
        
        # Если строка начинается с end_prefix и мы находимся в промежутке, заканчиваем собирать
        if line.startswith('Знание языков') and in_between:
            in_between = False
            continue  # Пропускаем строку с началом "end_prefix"
        elif line.startswith('Гражданство') and in_between:
            in_between = False
            continue  # Пропускаем строку с началом "end_prefix"
        
        # Если мы находимся в промежутке, добавляем строку
        if in_between:
            result.append(line)
        if lst == 'Не указано':
            return 'Не указано'
    
    return ",".join(result)



In [48]:
data['Учебные заведения'] = data["Учебные заведения"].apply(get_strings_between_start)

In [49]:
def find_between_strings_for_basic(lst, start_str, end_str):
    result = []
    found_start = False
    lst = lst.split(':')
    for items in lst:
        items = items.split('\n')
        for item in items:
            if found_start:
                if item == end_str:
                    break
                result.append(item)
            elif item == start_str:
                found_start = True
    if result == []:
        return 'Не указано'
    else:
        return result[:1]

In [50]:
start_words = ['Тип занятости', 'Сфера деятельности', 'Образование', 'Пол', 'Возраст']
for start_word in start_words: 
    data[start_word] = data.basic.apply(find_between_strings_for_basic, args=(start_word, None)).explode().str.strip()

In [51]:
data

,num,table,basic,description,Опыт работы,Стаж работы,Компания,Обязанности,Учебные заведения,Гражданство,Тип занятости,Сфера деятельности,Образование,Пол,Возраст
0,№ 3317133582,Учебные заведения\nГод окончания: 2014\nММК им...,"Тип занятости: Основная работа, Подработка\nСф...",Опыт работы в косметологии менее года,Не указано,0.00,Не указано,Не указано,"ММК им С.П. Боткина,Сестринское дело",Россия,"Основная работа, Подработка","Фитнес, салоны красоты",Среднее специальное,Женский,29 лет
1,№ 4585551392,Гражданство\nГражданство — Россия,Может выйти завтра: Нет\nТип занятости: Подраб...,,Не указано,0.00,Не указано,Не указано,Не указано,Россия,Подработка,"Фитнес, салоны красоты",Не указано,Женский,50 лет
2,№ 3628798315,Учебные заведения\nГод окончания: 2004\nБрянск...,Тип занятости: Основная работа\nГотовность к к...,"Эстeтичеcкая, инъекциoннaя, аппаратнaя коcмето...",Не указано,0.00,Не указано,Не указано,"Брянский базовый медицинский колледж,Сестринск...",Россия,Основная работа,"Фитнес, салоны красоты",Среднее специальное,Женский,38 лет
3,№ 4525922612,Учебные заведения\nГод окончания: 2008\nЧитинс...,Тип занятости: Подработка\nСфера деятельности:...,—,Не указано,0.00,Не указано,Не указано,"Читинский медицинский колледж,Сестринское дело",Россия,Подработка,"Фитнес, салоны красоты",Среднее специальное,Женский,37 лет
4,№ 4600063172,Учебные заведения\nГод окончания: 2024\nUltra ...,"Тип занятости: Основная работа, Подработка\nСф...","Mеня зoвут Анна, я нa дaнный момент работaю в ...",Не указано,0.00,Не указано,Не указано,"Ultra pro,Косметолог эстетист",Россия,"Основная работа, Подработка","Фитнес, салоны красоты",Среднее специальное,Женский,39 лет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,№ 4381095053,Опыт работы\n11 лет и 3 месяца\nОктябрь 2013 —...,Тип занятости: Подработка\nСфера деятельности:...,Рассматриваю дополнительный прием пациентов то...,"{year*11.25; company*Частная, Врач косметолог;...",11.25,"Частная, Врач косметолог",Инъекционная косметология от а до я .,"Мхитар Гераци . И.П. Павлова,Лечебное дело",Россия,Подработка,"Медицина, фармацевтика",Высшее,Женский,48 лет
1100,№ 3057007196,Опыт работы\n10 месяцев\nСентябрь 2022 — Июнь ...,Тип занятости: Основная работа\nГотовность к к...,Мнe вceгдa нрaвилось заниматься кoжей и eе здo...,"{year*0.83; company*Салон красоты «ЭКЗОТИКА», ...",0.83,"Салон красоты «ЭКЗОТИКА», Косметолог-эстетист","Эстетические процедуры ,такие как : чистки , п...","СГМУ ИМ.Разумовского,Сестринское дело",Россия,Основная работа,"Медицина, фармацевтика",Среднее специальное,Женский,22 года
1101,№ 2568568827,Опыт работы\n3 года и 2 месяца\nОктябрь 2019 —...,"Сфера деятельности: Фитнес, салоны красоты\nГр...","Коммуникабельная,ответственная,аккуратная и оч...","{year*3.17; company*Студия красоты «МАсква», В...",3.17,"Студия красоты «МАсква», Визажист-бровист,косм...",Оказание косметологических услуг клиентам: - К...,ТГМУ-Тверской государственный медицинский унив...,Россия,Не указано,"Фитнес, салоны красоты",Высшее,Женский,30 лет
1102,№ 4222300635,Учебные заведения\nГод окончания: 2023\nШкола ...,Может выйти завтра: Да\nТип занятости: Основна...,Меня зовут Светлана\nВ данный момент работаю в...,Не указано,0.00,Не указано,Не указано,"Школа Косметологии Ангелины Купцовой,Косметоло...",Россия,Основная работа,"Фитнес, салоны красоты",Среднее специальное,Женский,29 лет


## Переименование столбцов

In [52]:
data = data.rename(columns={"num": "Идентификатор на сайте", "description": "О себе"})

In [53]:
data = data.drop(['table', 'basic', 'Опыт работы'], axis = 1)

In [55]:
data.to_csv('my_data2.csv')